## Extracting Titanic Disaster Data From Kaggle

In [1]:
#We will directly load Kaggle credentials from .env file, hence we are importing python-dotenv.
# More info: https://stackoverflow.com/questions/41546883/can-somebody-explain-the-use-of-python-dotenv-module

!pip install python-dotenv

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
from dotenv import load_dotenv, find_dotenv

In [25]:
# find .env automatically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [26]:
# extracting environment variable using os.environ.get
import os
KAGGLE_USERNAME = os.environ.get("KAGGLE_USERNAME")
print(KAGGLE_USERNAME)

vjcalling


In [27]:
# imports
import requests
from requests import session
import os
from dotenv import load_dotenv, find_dotenv

In [50]:
from requests import session
# payload
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGLE_USERNAME"),
    'password': os.environ.get("KAGGLE_PASSWORD")
}


def extract_data(url, file_path):
    '''
    extract data from kaggle
    '''
    login_url = 'https://www.kaggle.com/account/login'
    # setup session
    with session() as c:
        c.post('https://www.kaggle.com/account/login', data=payload)
        # oppen file to write
        with open(file_path, 'wb') as handle:
            response = c.get(login_url).text
            #need to add antiForgeryToken for us to download data from Kagglw
            AFToken = response[response.index('antiForgeryToken') + 19:response.index('isAnonymous: ')-12]
            payload['__RequestVerificationToken']=AFToken
            c.post(login_url + "?isModal=true&returnUrl=/", data=payload)
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(block)


In [51]:
# urls
train_url = 'https://www.kaggle.com/c/titanic/download/train.csv'
test_url = 'https://www.kaggle.com/c/titanic/download/test.csv'

# file paths
raw_data_path = os.path.join(os.path.pardir,'data','raw')
train_data_path = os.path.join(raw_data_path, 'train.csv')
test_data_path = os.path.join(raw_data_path, 'test.csv')

# extract data
extract_data(train_url,train_data_path)
extract_data(test_url,test_data_path)



In [41]:
#checking if files are created

#Windows
!dir ..\\data\\raw

#Linux
#!ls -l ../data/raw

### Builiding the file script

In [43]:
get_raw_data_script_file = os.path.join(os.path.pardir,'src','data','get_raw_data.py')

In [46]:
%%writefile $get_raw_data_script_file
# -*- coding: utf-8 -*-
import os
from dotenv import find_dotenv, load_dotenv
from requests import session
import logging


# payload for login to kaggle
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGLE_USERNAME"),
    'password': os.environ.get("KAGGLE_PASSWORD")
}


def extract_data(url, file_path):
    '''
    extract data from kaggle
    '''
    login_url = 'https://www.kaggle.com/account/login'
    # setup session
    with session() as c:
        c.post('https://www.kaggle.com/account/login', data=payload)
        # oppen file to write
        with open(file_path, 'w') as handle:
            response = c.get(login_url).text
            #need to add antiForgeryToken for us to download data from Kagglw
            AFToken = response[response.index('antiForgeryToken') + 19:response.index('isAnonymous: ')-12]
            payload['__RequestVerificationToken']=AFToken
            c.post(login_url + "?isModal=true&returnUrl=/", data=payload)
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(str(block))


                
def main(project_dir):
    '''
    main method
    '''
    # get logger
    logger = logging.getLogger(__name__)
    logger.info('getting raw data')
    
    # urls
    train_url = 'https://www.kaggle.com/c/titanic/download/train.csv'
    test_url = 'https://www.kaggle.com/c/titanic/download/test.csv'

    # file paths
    raw_data_path = os.path.join(project_dir,'data','raw')
    train_data_path = os.path.join(raw_data_path, 'train.csv')
    test_data_path = os.path.join(raw_data_path, 'test.csv')

    # extract data
    extract_data(train_url,train_data_path)
    extract_data(test_url,test_data_path)
    logger.info('downloaded raw training and test data')


if __name__ == '__main__':
    # getting root directory
    project_dir = os.path.join(os.path.dirname(__file__), os.pardir, os.pardir)
    
    # setup logger
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)

    # find .env automatically by walking up directories until it's found
    dotenv_path = find_dotenv()
    # load up the entries as environment variables
    load_dotenv(dotenv_path)

    # call the main
    main(project_dir)


Overwriting ..\src\data\get_raw_data.py


In [47]:
!python $get_raw_data_script_file

2018-08-30 18:08:18,267 - __main__ - INFO - getting raw data
2018-08-30 18:08:32,118 - __main__ - INFO - downloaded raw training and test data
